In [77]:
from utils import read_lines

def parse_input(input_file: str) -> str:
    line: str = read_lines(input_file)[0]
    return line

minus = '..@@@@.'

plus = """
...@...
..@@@..
...@...
"""

reverse_l = """
....@..
....@..
..@@@..
"""

vertical_line = """
..@....
..@....
..@....
..@....
"""

square = """
..@@...
..@@...
"""

blocks = [minus, plus, reverse_l, vertical_line, square]
block_heights = [1, 3, 3, 4, 2]

def create_block(i):
    ans = []
    
    for line in blocks[i].split('\n'):
        if line:
            ans.append([c for c in line])
    return ans[::-1]

def create_gap():
    return [['.'] * 7 for _ in range(3)]

def find_block(game, bi):
    i = len(game) - 1
    while all(x != '@' for x in game[i]):
        i -= 1
    return i - block_heights[bi] + 1, i + 1 

def is_settled(game, bi):
    lo, hi = find_block(game, bi)
    if lo == 0:
        return True
    for i in range(lo, hi):
        for j in range(7):
            if game[i][j] == '@' and game[i-1][j] == '#':
                return True
    return False

def can_move_block(game, bi, wind):
    lo, hi = find_block(game, bi)
    if wind == '<':
        for i in range(lo, hi):
            for j in range(7):
                if game[i][j] == '@' and (j == 0 or game[i][j-1] == '#'):
                    return False
    else:
        for i in range(lo, hi):
            for j in range(7):
                if game[i][j] == '@' and (j == 6 or game[i][j+1] == '#'):
                    return False
    return True

def move_block(game, bi, wind):
    lo, hi = find_block(game, bi)
    if can_move_block(game, bi, wind):
        for i in range(lo, hi):
            if wind == '<':
                for j in range(6):
                    if game[i][j] == '.' and game[i][j+1] == '@':
                        game[i][j] = '@'
                        game[i][j+1] = '.'
            else:
                for j in range(6, 0, -1):
                    if game[i][j] == '.' and game[i][j-1] == '@':
                        game[i][j] = '@'
                        game[i][j-1] = '.'

def fall_block(game, bi):
    lo, hi = find_block(game, bi)
    for i in range(lo-1, hi-1):
        for j in range(7):
            if game[i][j] == '.' and game[i+1][j] == '@':
                game[i][j] = '@'
                game[i+1][j] = '.'
    if all(x == '.' for x in game[-1]):
        game.pop()

def settle_block(game, bi):
    lo, hi = find_block(game, bi)
    for i in range(lo, hi):
        for j in range(7):
            if game[i][j] == '@':
                game[i][j] = '#'

def print_game(game, i):
    print('turn: ', i + 1)
    for i in range(len(game) - 1, -1, -1):
        print(''.join(game[i]))
    print()

def part1(input_file, cnt_blocks):
    winds = parse_input(input_file)
    game = []
    turn = 0
    for i in range(cnt_blocks):
        bi = i % len(blocks)
        game += create_gap() + create_block(bi)
        while True:
            wind = winds[turn % len(winds)]
            turn += 1
            move_block(game, bi, wind)
            if is_settled(game, bi):
                break
            fall_block(game, bi)
            
        settle_block(game, bi)
        # print_game(game, i)
    return len(game)
        

In [79]:
part1('inputs/day17.txt', 2022)

3188

In [112]:
def predict(heights, block_cnt):
    n = len(heights)
    deltas = []
    for i in range(n - 1):
        deltas.append(heights[i + 1] - heights[i])

    p_start = 0
    p_len = 0
    patterns = {}
    for i in range(n - 8):
        p = tuple(deltas[i: i+30])
        if p in patterns:
            p_start = patterns[p]
            p_len = i-patterns[p]
            print(patterns[p], i-patterns[p])
            break
        else:
            patterns[p] = i

    p_growth = sum(deltas[p_start: p_start + p_len])
    cnt_p = (block_cnt - p_start) // p_len
    remain = (block_cnt - p_start) % p_len
    return heights[p_start] + cnt_p * p_growth + sum(deltas[p_start: p_start + remain])

def part2(input_file, block_cnt):
    winds = parse_input(input_file)
    game = []
    turn = 0
    heights = [0]
    for i in range(2022):
        bi = i % len(blocks)
        game += create_gap() + create_block(bi)
        while True:
            wind = winds[turn % len(winds)]
            turn += 1
            move_block(game, bi, wind)
            if is_settled(game, bi):
                break
            fall_block(game, bi)
            
        settle_block(game, bi)
        heights.append(len(game))
        # print_game(game, i)
    return predict(heights, block_cnt)

In [113]:
block_cnt = 1000000000000
# block_cnt = 2022
part2('inputs/day17.txt', block_cnt)

54 1745


1591977077342